In [1]:
print("all ok")

all ok


In [4]:
from langchain_community.document_loaders import PyPDFLoader

attention_paper_loader=PyPDFLoader(r'..\resource\research_paper\attention_all_you_need.pdf')
lossmiddle_paper_loader=PyPDFLoader(r'..\resource\research_paper\loss_in_middle.pdf')

attention_data=attention_paper_loader.load()
lossmiddle_data=lossmiddle_paper_loader.load()

In [5]:
len(attention_data)

15

In [6]:
len(lossmiddle_data)

18

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [8]:
attention_splitted_text=splitter.split_documents(attention_data)
lossmiddle_splitted_text=splitter.split_documents(lossmiddle_data)

In [9]:
len(attention_splitted_text)

93

In [10]:
len(lossmiddle_splitted_text)

153

### Load Embedding model

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model='text-embedding-ada-002')

### Datastore loading

In [40]:
from langchain_chroma import Chroma

attention_vectorstore=Chroma(collection_name="attention_datastore",
                   embedding_function=embeddings,
                   persist_directory=r'..\Advance_Rag\dbstore\attention')

In [41]:
lossmiddle_vectorstore=Chroma(collection_name="lossmiddle_datastore",
                   embedding_function=embeddings,
                   persist_directory=r'..\Advance_Rag\dbstore\lossmiddle')

In [42]:
attention_vectorstore.add_documents(attention_splitted_text)
lossmiddle_vectorstore.add_documents(lossmiddle_splitted_text)

['ef302a91-2440-40aa-9fd1-2f92907336cc',
 '39d5b5f8-f5d5-495b-9012-c7b38687f7ac',
 'ff30dd04-8e40-44e1-8478-b5f62e23e9c7',
 '6d3bef18-0128-433e-b2ed-6f36ad79cbb8',
 '7636d6e3-488e-489b-9c54-f1604dc75be7',
 '0e2b3188-db89-49e2-9c3a-f29fec3d9bd9',
 '5e781ac1-fdde-4376-b97c-931b07c65aec',
 '3397a16b-b3a4-4cc7-958e-bbcefc633166',
 '4bb5af16-6ddb-4345-aaa5-7ea25992a671',
 '542cbf52-55c8-4583-956f-94787c4dd22f',
 '8eb29832-60a0-48b0-8857-5fb8a253a96c',
 '89d778bd-57c2-4e38-9d3a-c591cbfe32ff',
 '2a44883f-1e1f-4977-92ae-2000c86e11a8',
 'dc07f706-94b5-48bd-a5a9-f8c33198dcb1',
 '3ae6ae2f-28e3-4339-966a-022b742988c7',
 'e76e2987-0b1f-43cf-9a30-b926c0771cb4',
 '4990742e-3d75-4800-a261-e7a7abe9146d',
 '9ddf3ddc-054b-4eaf-a66e-e3bc354521f0',
 '29f8210c-24b3-4e2c-99da-996f59729a7c',
 '09c97f4b-1925-4e08-abc9-961a1d1ef89c',
 'bb1fd846-a85f-4e20-8559-ce89b277ad8d',
 'aebda6cf-6431-40e4-9622-38af6419d0c9',
 '0e7ef8d4-21e9-480f-bb9d-da64322b9393',
 'bf374dc9-b3ec-453f-a19b-0f6b874f3794',
 '793657eb-67b2-

In [43]:
attention_retriever=attention_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})
lossmiddle_retriever=lossmiddle_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

#### Merge two retriever

In [44]:
from langchain.retrievers.merger_retriever import MergerRetriever

merged_retriever=MergerRetriever(retrievers=[attention_retriever,lossmiddle_retriever])

In [46]:
for chunks in merged_retriever.get_relevant_documents("What is transformer?"):
    print(chunks)

page_content='language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].
3 Model Architecture' metadata={'source': '..\\resource\\research_paper\\attention_all_you_need.pdf', 'title': '', 'keywords': '', 'page_label': '2', 'creationdate': '2024-04-10T21:11:43+00:00', 'subject': '', 'trapped': '/False', 'moddate': '2024-04-10T21:11:43+00:00', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'page': 1, 'total_pages': 15, 'author': '', 'creator': 'LaTeX with hyperref'}
page_content='There is much prior work in designing performant
language models with cheaper 

In [47]:
for chunks in merged_retriever.get_relevant_documents("What is loss of middle phenomena?"):
    print(chunks)

page_content='The third is the path length between long-range dependencies in the network. Learning long-range
dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the
ability to learn such dependencies is the length of the paths forward and backward signals have to
traverse in the network. The shorter these paths between any combination of positions in the input
and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare' metadata={'keywords': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'moddate': '2024-04-10T21:11:43+00:00', 'page': 5, 'title': '', 'subject': '', 'creator': 'LaTeX with hyperref', 'page_label': '6', 'author': '', 'source': '..\\resource\\research_paper\\attention_all_you_need.pdf', 'producer': 'pdfTeX-1.40.25', 'creationdate': '2024-04-10T21:11:43+00:00', 'total_pages': 15, 'trapped': '/False'}
page_content='formance beg

### Response from merged retriever is too much messy apply lost in middle techniques

In [48]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder
from re import search
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"]= os.getenv("GOOGLE_API_KEY")

In [49]:
filter=EmbeddingsRedundantFilter(embeddings=embeddings)
reordering=LongContextReorder()

pipeline=DocumentCompressorPipeline(transformers=[filter,reordering])
compression_retriever_reorder=ContextualCompressionRetriever(
    base_compressor=pipeline,base_retriever=merged_retriever,
    search_kwargs={"k": 3, "include_metadata": True}
)

In [58]:
compression_retriever_reorder.invoke("What is transformers?")

[_DocumentWithState(metadata={'producer': 'pdfTeX-1.40.25', 'title': '', 'trapped': '/False', 'page': 9, 'total_pages': 18, 'subject': '', 'author': '', 'keywords': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with hyperref', 'page_label': '10', 'source': '..\\resource\\research_paper\\loss_in_middle.pdf', 'moddate': '2023-11-22T01:09:40+00:00', 'creationdate': '2023-11-22T01:09:40+00:00'}, page_content='There is much prior work in designing performant\nlanguage models with cheaper scaling than Trans-\nformers in the context length. Many lines of work\npursue Transformer variants with attention modi-\nfications like recurrence (Dai et al., 2019), factor-\nizing attention into computationally less intensive\napproximations (Beltagy et al., 2020; Zaheer et al.,\n2020), or low-rank approximations (Wang et al.,\n2020; Peng et al., 2021). Dao et al. (2022) in-', state={'embedded_doc': [-0.01343067176640033

### Loading LLM

In [51]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-pro')

In [52]:
res=llm.invoke("How are you")
res.content

"Thanks for asking! As an AI, I don't have feelings, but I'm functioning perfectly and I'm ready to help.\n\nHow can I assist you today?"

In [62]:
from langchain_openai import ChatOpenAI
openai_llm=ChatOpenAI(model='gpt-4o')

print(openai_llm.invoke("How are you?"))

content="Thank you for asking! I'm just a program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 11, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CBd8ZXMyUiXR5Pg2cSn2g8H0PpbYo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--794e9a8b-7a61-4a19-a75a-c044d3bfd79b-0' usage_metadata={'input_tokens': 11, 'output_tokens': 36, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### RAG chain

In [63]:
from langchain.chains import RetrievalQA

qa_chain=RetrievalQA.from_chain_type(
    llm=openai_llm,
    chain_type='stuff',
    retriever=compression_retriever_reorder,
    return_source_documents=True

)

In [64]:
query="What is transformers?"
response=qa_chain(query)
print(response['result'])

Transformers are a type of neural network architecture used primarily for handling sequence-to-sequence tasks, like translation, language modeling, and other tasks involving sequences of data. Introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017, Transformers rely entirely on self-attention mechanisms to calculate and utilize the dependencies between input and output data sequences, without the use of recurrent or convolutional neural networks. This allows them to process sequences in parallel, which significantly improves computational efficiency and enables them to scale to larger datasets more effectively. They have become the foundation for many state-of-the-art models in natural language processing.


#### LCEL process

In [65]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

template="""You are a helpful assistant. Answer the question based on context:

Context: {context}
Question: {query}
Answer:
"""

prompt=PromptTemplate.from_template(
    template=template
)


In [68]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [69]:
lcel_chain=({"context":compression_retriever_reorder | format_docs,"query": RunnablePassthrough()}
            | prompt
            | openai_llm
            | StrOutputParser()
            )

In [ ]:
response=lcel_chain.invoke("What is transformer?")
response

'The Transformer is a type of neural network architecture introduced for sequence transduction tasks, such as language translation, which relies entirely on self-attention mechanisms to compute representations of its input and output. Unlike previous models based on recurrent or convolutional neural networks, the Transformer employs self-attention to enhance parallelization during training, allowing it to achieve state-of-the-art results in tasks like translation with reduced sequential computation. This architecture enables the Transformer to be more efficient and effective, achieving significant performance improvements when trained on high-performance hardware like GPUs.'

In [71]:
response=lcel_chain.invoke("What is loss-middle phenomena?")
response

'The "loss-middle phenomenon" refers to a performance degradation pattern observed when relevant information is placed in the middle of a sequence or context within language models. Specifically, models tend to show better performance when relevant information is positioned at the beginning or the end of a context, rather than the middle. This results in a U-shaped performance curve, where accuracy is highest for information located at the start and end, due in part to the serial-position effect observed in human memory, where individuals are known to recall the first and last elements of a list better than those in the middle.'

### Merged retriever response

In [72]:
lcel_chain_merged=({"context":merged_retriever | format_docs,"query": RunnablePassthrough()}
            | prompt
            | openai_llm
            | StrOutputParser()
            )

In [73]:
response=lcel_chain_merged.invoke("What is transformer?")
response

'The Transformer is a type of neural network architecture that relies entirely on self-attention mechanisms to compute representations of its input and output. It does not use sequence-aligned recurrent neural networks (RNNs) or convolutional networks, which were the dominant sequence transduction models before Transformers were introduced. The architecture allows for significantly more parallelization and can achieve high performance in tasks like translation after relatively short training times. The Transformer model was proposed as a simpler alternative to complex recurrent or convolutional neural networks and includes innovations such as scaled dot-product attention and multi-head attention.'